<a href="https://colab.research.google.com/github/HektorLin/indexliquidity/blob/main/Illquidity_Contagion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import pickle
pd.set_option('display.float_format', lambda x: '%.6f' % x)
import numpy as np
import seaborn as sns
import gc

from datetime import datetime, date
!pip install icecream
from icecream import ic
from scipy.stats import ttest_1samp

import matplotlib.pyplot as plt
%matplotlib inline
import statsmodels.api as sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Functions

In [ ]:
def winsorization (vector, threshold):
    vector = vector.clip(lower=vector.quantile(threshold), upper=vector.quantile(1-threshold))
    return vector

In [ ]:
def wavg(group, avg_name, weight_name):
  d = group[avg_name]
  w = group[weight_name]
  return (d * w).sum() / w.sum()

# Data Preparation


Get the sample of ETFs

1. Identify ETFs by CRSP ETF flag in CRSP MF Fund Summary file (2010-2021)
2. Choose only index_flag==D + objective == ED + valid tickers ('crsp_fundno','crsp_portno','ncusip','ticker')
3. Require TNA above period *median*

In [ ]:
# Identify ETFs by CRSP ETF flag in CRSP MF Fund Summary file
ETFlist = pd.read_csv('/content/drive/My Drive/Projects/Illiquidity_Contagion/Data/Raw/ETFlist_CRSP.csv',dtype={'crsp_fundno':str,'crsp_portno':str})
ETFlist

,ticker,crsp_fundno,summary_period2,caldt,nav_latest,tna_latest,per_com,per_pref,per_cash,crsp_portno,...,retail_fund,inst_fund,index_fund_flag,et_flag,first_offer_dt,dead_flag,exp_ratio,turn_ratio,crsp_obj_cd,lipper_obj_cd
0,ADRA,006679,AQ,31DEC2019,34.535400,19.000000,0.000000,0.000000,0.060000,1026749,...,N,Y,D,F,13NOV2002,Y,NaN,NaN,EFRP,PC
1,ADRA,006679,Q,29MAR2019,31.413500,18.800000,0.000000,0.000000,0.130000,1026749,...,N,Y,B,F,13NOV2002,Y,0.003000,0.200000,EFRP,PC
2,ADRA,006679,Q,28SEP2018,32.568200,19.500000,0.000000,0.000000,0.670000,1026749,...,N,Y,B,F,13NOV2002,Y,0.003000,0.090000,EFRP,PC
3,ADRA,006679,AQ,31DEC2018,28.400400,17.000000,0.000000,0.000000,0.200000,1026749,...,N,Y,B,F,13NOV2002,Y,0.003000,0.200000,EFRP,PC
4,ADRA,006679,Q,28JUN2019,30.834900,18.500000,0.000000,0.000000,0.080000,1026749,...,N,Y,B,F,13NOV2002,Y,0.003000,0.200000,EFRP,PC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61414,SAEF,099209,AQ,31DEC2021,24.433900,8.200000,NaN,NaN,NaN,1052956,...,N,Y,NaN,F,15NOV2021,N,NaN,NaN,EDYG,G
61415,UVDV,099217,AQ,31DEC2021,10.174500,8.400000,NaN,NaN,NaN,1052974,...,N,Y,NaN,F,18NOV2021,N,NaN,NaN,EDYI,EI
61416,TYNE,099218,AQ,31DEC2021,23.533000,2.400000,89.800000,0.000000,0.130000,1052975,...,N,Y,D,F,23NOV2021,N,NaN,NaN,EDSI,ID
61417,IBET,099227,AQ,31DEC2021,13.162400,0.300000,NaN,NaN,NaN,1052897,...,N,Y,NaN,F,15NOV2021,N,NaN,NaN,EDSA,TL


In [ ]:
ETFlist[ETFlist.ticker=='HON']

,ticker,crsp_fundno,summary_period2,caldt,nav_latest,tna_latest,per_com,per_pref,per_cash,crsp_portno,...,retail_fund,inst_fund,index_fund_flag,et_flag,first_offer_dt,dead_flag,exp_ratio,turn_ratio,crsp_obj_cd,lipper_obj_cd


In [ ]:
len(ETFlist.crsp_fundno.unique()),len(ETFlist.crsp_portno.unique()),len(ETFlist.ncusip.unique()),len(ETFlist.ticker.unique()),len(ETFlist.fund_name.unique())

(2593, 2977, 3276, 2739, 4481)

In [ ]:
# objective == ED (equity domestic)
subset = ETFlist.loc[(ETFlist.crsp_obj_cd.str[0:2]=='ED') & (ETFlist.crsp_portno.isnull()==False),:]
len(subset.crsp_fundno.unique()),len(subset.crsp_portno.unique()),len(subset.ncusip.unique()),len(subset.ticker.unique()),len(subset.fund_name.unique())

(1725, 1990, 2233, 1815, 2879)

In [ ]:
# require valid tickers
subset = subset.dropna(subset=['crsp_fundno','crsp_portno','ncusip','ticker'],how='any',axis=0)
len(subset.crsp_fundno.unique()),len(subset.crsp_portno.unique()),len(subset.ncusip.unique()),len(subset.ticker.unique()),len(subset.fund_name.unique())

(1725, 1990, 2230, 1814, 2867)

In [ ]:
# require valid NAV and TNA
subset = subset.dropna(subset=['nav_latest','tna_latest'],how='any',axis=0)
len(subset.crsp_fundno.unique()),len(subset.crsp_portno.unique()),len(subset.ncusip.unique()),len(subset.ticker.unique()),len(subset.fund_name.unique())

(1703, 1947, 2208, 1798, 2843)

In [ ]:
# set up time index
subset['caldt'] = pd.to_datetime(subset['caldt'],dayfirst=True)
subset['first_offer_dt'] = pd.to_datetime(subset['first_offer_dt'],dayfirst=True)

subset.sort_values(by=['caldt','crsp_fundno'],inplace=True)
subset = subset.reset_index(drop=True)

In [ ]:
# get the median TNA
temp = subset[["caldt","tna_latest"]].groupby(["caldt"]).quantile(0.5).reset_index()
temp.columns = ["caldt","median_tna"]
temp

,caldt,median_tna
0,2010-03-31,97.400000
1,2010-06-30,74.950000
2,2010-09-30,95.800000
3,2010-12-31,106.350000
4,2011-03-31,122.600000
5,2011-06-30,81.300000
6,2011-09-30,58.200000
7,2011-12-30,58.800000
8,2012-03-30,72.450000
9,2012-06-29,60.200000


In [ ]:
# drop observations below median TNA
subset = subset.merge(temp,how='left',on='caldt')
subset = subset.loc[(subset.tna_latest>=subset.median_tna),:]
len(subset.crsp_fundno.unique()),len(subset.crsp_portno.unique()),len(subset.ncusip.unique()),len(subset.ticker.unique()),len(subset.fund_name.unique())

(753, 840, 970, 791, 1264)

In [ ]:
# get lookup keys
subset['crsp_portno'].drop_duplicates().to_csv('crsp_portno.txt',index=False,header=False)
subset['crsp_fundno'].drop_duplicates().to_csv('crsp_fundno.txt',index=False,header=False)
subset['ncusip'].drop_duplicates().to_csv('ncusip.txt',index=False,header=False)
subset['ticker'].drop_duplicates().to_csv('ticker.txt',index=False,header=False)

Get trading information for the sampled ETFs 

In [ ]:
# retrived using ticker
CRSPetf = pd.read_csv('/content/drive/My Drive/Projects/Illiquidity_Contagion/Data/Raw/ETFdaily_CRSP.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (4,11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
# remove non-numeric returns
CRSPetf.drop(labels=CRSPetf[CRSPetf.RET.isin(['B','C'])].index,axis=0,inplace=True)
CRSPetf.RET = CRSPetf.RET.astype(float)

# set time index
CRSPetf.date = pd.to_datetime(CRSPetf.date,dayfirst=True)
CRSPetf['yearMonth'] = CRSPetf.date.apply(lambda x: x.strftime('%Y%m')).astype(int)
CRSPetf['yearWeek'] = CRSPetf.date.dt.strftime('%Y-%V')

In [ ]:
CRSPetf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1793598 entries, 1 to 1794447
Data columns (total 18 columns):
 #   Column     Dtype         
---  ------     -----         
 0   PERMNO     int64         
 1   date       datetime64[ns]
 2   SHRCD      float64       
 3   EXCHCD     float64       
 4   NCUSIP     object        
 5   TICKER     object        
 6   NAICS      float64       
 7   BIDLO      float64       
 8   ASKHI      float64       
 9   PRC        float64       
 10  VOL        float64       
 11  RET        float64       
 12  BID        float64       
 13  ASK        float64       
 14  SHROUT     float64       
 15  OPENPRC    float64       
 16  yearMonth  int64         
 17  yearWeek   object        
dtypes: datetime64[ns](1), float64(12), int64(2), object(3)
memory usage: 260.0+ MB


In [ ]:
CRSPetf.drop(CRSPetf[CRSPetf.SHRCD!=73].index,axis=0,inplace=True)

In [ ]:
CRSPetf['PRC'] = np.abs(CRSPetf['PRC'])
CRSPetf['OPENPRC'] = np.abs(CRSPetf['OPENPRC'])

# Bid-Ask-spread = the quoted spread / mid-point
CRSPetf['BAS'] = (CRSPetf['ASK'] - CRSPetf['BID']) / (CRSPetf['ASK'] + CRSPetf['BID']) * 2 * 100
CRSPetf['BAS'] = np.where(CRSPetf['BAS']<0,-CRSPetf['BAS'],CRSPetf['BAS'])

# Amihud = abs of daily return / dollar trading volume: how much returns in percentage points can 1 million push
CRSPetf['Amihud'] = np.abs(CRSPetf['RET']*100) / (CRSPetf['VOL'] * CRSPetf['PRC'] /1000000)
CRSPetf['OCAmihud'] = np.abs((CRSPetf['OPENPRC']/CRSPetf['PRC']-1)*100) / (CRSPetf['VOL'] * CRSPetf['PRC'] /1000000)

# turnover
CRSPetf['turnover'] = CRSPetf['VOL'] / (CRSPetf['SHROUT']*1000) * 100
CRSPetf['dvol'] = CRSPetf['VOL']*CRSPetf['PRC'] / 1000000 # dollar trading volume in million

# size in millions
CRSPetf['size'] = CRSPetf['SHROUT']/1000 * CRSPetf['PRC']

In [ ]:
CRSPetf.drop(CRSPetf[CRSPetf.VOL==0].index,axis=0, inplace=True)

In [ ]:
CRSPetf = CRSPetf[['TICKER','yearMonth','yearWeek','BAS','Amihud','OCAmihud','turnover','dvol','size','RET']].groupby(['TICKER','yearWeek']).median().reset_index()

In [ ]:
CRSPetf[['BAS','Amihud','OCAmihud']].describe(percentiles = [0.005,0.01,0.02,0.98,0.99,0.995])

,BAS,Amihud,OCAmihud
count,356035.000000,356035.000000,355952.000000
mean,0.126202,9.027887,1.204386
std,0.235527,685.299517,5.336783
min,0.000000,0.000000,0.000000
0.5%,0.005879,0.000192,0.000000
1%,0.007591,0.000408,0.000116
2%,0.009961,0.000840,0.000457
50%,0.080970,0.239626,0.183739
98%,0.547883,25.511283,9.908752
99%,0.723153,51.399757,15.591251


In [ ]:
CRSPetf['BASdiff'] =  CRSPetf.groupby(['TICKER']).BAS.diff()

CRSPetf['lnAdiff'] = np.log(CRSPetf['Amihud']+1)
CRSPetf['lnAdiff'] =  CRSPetf.groupby(['TICKER']).lnAdiff.diff()

CRSPetf['lnOCAdiff'] = np.log(CRSPetf['OCAmihud']+1)
CRSPetf['lnOCAdiff'] =  CRSPetf.groupby(['TICKER']).lnOCAdiff.diff()

CRSPetf.dropna(axis=0, inplace=True)

In [ ]:
CRSPetf['year'] = CRSPetf.yearWeek.str[0:4].astype(int)
CRSPetf.drop(CRSPetf[CRSPetf.year<=2010].index,inplace=True,axis=0)

In [ ]:
CRSPetf.to_pickle('/content/drive/My Drive/Projects/Illiquidity_Contagion/Data/Raw/CRSPetf_week.pkl')

In [ ]:
with open('/content/drive/My Drive/Projects/Illiquidity_Contagion/Data/Raw/CRSPetf_week.pkl', 'rb') as f:
    CRSPetf = pickle.load(f)

In [ ]:
CRSPetf.loc[CRSPetf.yearWeek=='2020-21',('TICKER','size','BAS')].sort_values(by='BAS',ascending = True).head(50)

,TICKER,size,BAS
282008,SPY,266645.190000,0.003383
322917,VOO,134549.739797,0.003730
35764,DIA,19489.625860,0.004072
224675,QQQ,107920.296000,0.004396
113737,IJH,41220.255000,0.006053
124225,IVV,185913.126280,0.006754
128614,IWM,36273.800000,0.007464
346277,XLY,12285.881940,0.008349
345599,XLV,24833.947950,0.009998
343145,XLK,28510.893840,0.010403


In [ ]:
CRSPetf[['BASdiff','lnAdiff']].describe(percentiles = [0.001,0.01,0.02,0.98,0.99,0.999])

,BASdiff,lnAdiff
count,334941.000000,334941.000000
mean,-0.000196,-0.001353
std,0.167871,0.466618
min,-27.944866,-8.384683
0.1%,-0.719905,-3.043482
1%,-0.197993,-1.465539
2%,-0.128500,-1.109318
50%,-0.000077,-0.000012
98%,0.128844,1.092675
99%,0.196154,1.438633


In [ ]:
CRSPetf.loc[:,['BASdiff','lnAdiff','lnOCAdiff']].corr()

,BASdiff,lnAdiff,lnOCAdiff
BASdiff,1.000000,0.029496,0.014831
lnAdiff,0.029496,1.000000,0.300563
lnOCAdiff,0.014831,0.300563,1.000000


In [ ]:
CRSPetf.loc[CRSPetf.TICKER=='SPY',['BAS','Amihud','OCAmihud']].corr()

,BAS,Amihud,OCAmihud
BAS,1.000000,0.197645,0.192929
Amihud,0.197645,1.000000,0.534621
OCAmihud,0.192929,0.534621,1.000000


In [ ]:
CRSPetf.loc[CRSPetf.TICKER=='SPY',['BASdiff','lnAdiff','lnOCAdiff']].corr()

,BASdiff,lnAdiff,lnOCAdiff
BASdiff,1.000000,0.047679,0.011989
lnAdiff,0.047679,1.000000,0.402400
lnOCAdiff,0.011989,0.402400,1.000000


In [ ]:
CRSPetf.loc[CRSPetf.TICKER=='SPY',['BAS','Amihud','BASdiff','lnAdiff']]

,BAS,Amihud,BASdiff,lnAdiff
281519,0.007866,0.000013,-0.000085,0.000003
281520,0.007784,0.000025,-0.000082,0.000013
281521,0.007785,0.000012,0.000001,-0.000014
281522,0.007735,0.000021,-0.000050,0.000010
281523,0.007647,0.000016,-0.000089,-0.000005
...,...,...,...,...
282088,0.002208,0.000026,0.000071,0.000017
282089,0.002138,0.000026,-0.000070,-0.000000
282090,0.002145,0.000017,0.000007,-0.000009
282091,0.002178,0.000030,0.000032,0.000013


Get ETF stock holdings

In [ ]:
# retrived using crsp_portno
ETFholdings = pd.read_csv('/content/drive/My Drive/Projects/Illiquidity_Contagion/Data/Raw/ETFholdings_CRSP.csv',dtype={'permno':str,'crsp_portno':str})
#ETFholdings.drop('eff_dt',axis=1,inplace=True)

In [ ]:
# lookup keys
ETFholdings.dropna(subset=['permno'],axis=0,inplace=True)
ETFholdings.permno.drop_duplicates().to_csv('port_permno.txt',index=False,header=False)

Get stock trading information

In [ ]:
save=[]
for i in range(0,10):
    save.append([i*j for j in range(20,30)])

In [ ]:
save

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
 [40, 42, 44, 46, 48, 50, 52, 54, 56, 58],
 [60, 63, 66, 69, 72, 75, 78, 81, 84, 87],
 [80, 84, 88, 92, 96, 100, 104, 108, 112, 116],
 [100, 105, 110, 115, 120, 125, 130, 135, 140, 145],
 [120, 126, 132, 138, 144, 150, 156, 162, 168, 174],
 [140, 147, 154, 161, 168, 175, 182, 189, 196, 203],
 [160, 168, 176, 184, 192, 200, 208, 216, 224, 232],
 [180, 189, 198, 207, 216, 225, 234, 243, 252, 261]]

In [ ]:
# retrived using PERMNO
CRSPstock = pd.read_csv('/content/drive/My Drive/Projects/Illiquidity_Contagion/Data/Raw/stockdaily_CRSP.csv',dtype={'PERMNO':str})

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (4,11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
CRSPstock.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15786535 entries, 0 to 15786534
Data columns (total 16 columns):
 #   Column   Dtype  
---  ------   -----  
 0   PERMNO   object 
 1   date     object 
 2   SHRCD    float64
 3   EXCHCD   float64
 4   NCUSIP   object 
 5   TICKER   object 
 6   NAICS    float64
 7   BIDLO    float64
 8   ASKHI    float64
 9   PRC      float64
 10  VOL      float64
 11  RET      object 
 12  BID      float64
 13  ASK      float64
 14  SHROUT   float64
 15  OPENPRC  float64
dtypes: float64(11), object(5)
memory usage: 1.9+ GB


In [ ]:
# remove non-numeric returns
CRSPstock.drop(labels=CRSPstock[CRSPstock.RET.isin(['B','C'])].index,axis=0,inplace=True)
CRSPstock.RET = CRSPstock.RET.astype(float)

# set time index
CRSPstock.date = pd.to_datetime(CRSPstock.date,dayfirst=True)
CRSPstock['yearMonth'] = CRSPstock.date.apply(lambda x: x.strftime('%Y%m')).astype(int)
CRSPstock['yearWeek'] = CRSPstock.date.dt.strftime('%Y-%V')

In [ ]:
CRSPstock.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15632727 entries, 0 to 15786534
Data columns (total 18 columns):
 #   Column     Dtype         
---  ------     -----         
 0   PERMNO     object        
 1   date       datetime64[ns]
 2   SHRCD      float64       
 3   EXCHCD     float64       
 4   NCUSIP     object        
 5   TICKER     object        
 6   NAICS      float64       
 7   BIDLO      float64       
 8   ASKHI      float64       
 9   PRC        float64       
 10  VOL        float64       
 11  RET        float64       
 12  BID        float64       
 13  ASK        float64       
 14  SHROUT     float64       
 15  OPENPRC    float64       
 16  yearMonth  int64         
 17  yearWeek   object        
dtypes: datetime64[ns](1), float64(12), int64(1), object(4)
memory usage: 2.2+ GB


In [ ]:
CRSPstock.describe()

,SHRCD,EXCHCD,NAICS,BIDLO,ASKHI,PRC,VOL,RET,BID,ASK,SHROUT,OPENPRC,yearMonth
count,15632290.000000,15632290.000000,15460877.000000,15626149.000000,15626149.000000,15626149.000000,15626136.000000,15626149.000000,15626055.000000,15626045.000000,15629555.000000,15417862.000000,15632727.000000
mean,16.818746,2.130754,435664.802296,78.424761,79.919156,78.620935,1337640.171709,0.000625,79.125326,79.256069,129565.851295,79.975249,201558.836426
std,14.871218,1.010751,123969.891274,3399.976453,3443.859520,3422.049567,6604716.537949,0.038722,3420.611483,3424.512046,427417.777373,3445.900252,345.733477
min,11.000000,-2.000000,108100.000000,0.000100,0.001400,-1348.540040,0.000000,-0.965248,0.000600,0.000700,9.000000,0.000100,200912.000000
25%,11.000000,1.000000,332999.000000,6.990000,7.280000,6.790000,53907.000000,-0.011609,7.100000,7.150000,18500.000000,7.150000,201301.000000
50%,11.000000,3.000000,488390.000000,17.160000,17.720000,17.230000,236138.000000,0.000000,17.400000,17.480000,42055.000000,17.590000,201601.000000
75%,12.000000,3.000000,525990.000000,39.010000,40.080000,39.370000,874200.000000,0.011628,39.520000,39.610000,101053.000000,39.850000,201901.000000
max,89.000000,33.000000,999990.000000,452545.125000,455910.000000,454550.000000,2144903939.000000,12.365155,452700.031300,453384.000000,29206400.000000,453727.656250,202112.000000


In [ ]:
CRSPstock.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15632727 entries, 0 to 15786534
Data columns (total 18 columns):
 #   Column     Dtype         
---  ------     -----         
 0   PERMNO     object        
 1   date       datetime64[ns]
 2   SHRCD      float64       
 3   EXCHCD     float64       
 4   NCUSIP     object        
 5   TICKER     object        
 6   NAICS      float64       
 7   BIDLO      float64       
 8   ASKHI      float64       
 9   PRC        float64       
 10  VOL        float64       
 11  RET        float64       
 12  BID        float64       
 13  ASK        float64       
 14  SHROUT     float64       
 15  OPENPRC    float64       
 16  yearMonth  int64         
 17  yearWeek   object        
dtypes: datetime64[ns](1), float64(12), int64(1), object(4)
memory usage: 2.2+ GB


In [ ]:
# cross-section variables

CRSPstock['PRC'] = np.abs(CRSPstock['PRC'])
CRSPstock['OPENPRC'] = np.abs(CRSPstock['OPENPRC'])

# Bid-Ask-spread = the quoted spread / mid-point
CRSPstock['BAS'] = (CRSPstock['ASK'] - CRSPstock['BID']) / (CRSPstock['ASK'] + CRSPstock['BID']) * 2 * 100
CRSPstock['BAS'] = np.where(CRSPstock['BAS']<0,-CRSPstock['BAS'],CRSPstock['BAS'])

# Amihud = abs of daily return / dollar trading volume: how much returns in percentage points can 1 million push
CRSPstock['Amihud'] = np.abs(CRSPstock['RET']*100) / (CRSPstock['VOL'] * CRSPstock['PRC'] /1000000)
CRSPstock['OCAmihud'] = np.abs((CRSPstock['OPENPRC']/CRSPstock['PRC']-1)*100) / (CRSPstock['VOL'] * CRSPstock['PRC'] /1000000)

# turnover 
CRSPstock['turnover'] = CRSPstock['VOL'] / (CRSPstock['SHROUT']*1000) * 100
CRSPstock['dvol'] = CRSPstock['VOL']*CRSPstock['PRC'] / 1000000 # dollar trading volume in million

# size in millions
CRSPstock['size'] = CRSPstock['SHROUT']/1000 * CRSPstock['PRC']
CRSPstock['lnsize'] = np.log(CRSPstock['size']+1)

# string NASIC
#CRSPstock['NAICS'] = CRSPstock['NAICS'].astype(int).aystpe(str)

In [ ]:
CRSPstock[['BAS','Amihud','OCAmihud']].corr()

,BAS,Amihud,OCAmihud
BAS,1.000000,0.072678,0.188770
Amihud,0.072678,1.000000,0.160991
OCAmihud,0.188770,0.160991,1.000000


In [ ]:
CRSPstock.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15632727 entries, 0 to 15786534
Data columns (total 25 columns):
 #   Column     Dtype         
---  ------     -----         
 0   PERMNO     object        
 1   date       datetime64[ns]
 2   SHRCD      float64       
 3   EXCHCD     float64       
 4   NCUSIP     object        
 5   TICKER     object        
 6   NAICS      float64       
 7   BIDLO      float64       
 8   ASKHI      float64       
 9   PRC        float64       
 10  VOL        float64       
 11  RET        float64       
 12  BID        float64       
 13  ASK        float64       
 14  SHROUT     float64       
 15  OPENPRC    float64       
 16  yearMonth  int64         
 17  yearWeek   object        
 18  BAS        float64       
 19  Amihud     float64       
 20  OCAmihud   float64       
 21  turnover   float64       
 22  dvol       float64       
 23  size       float64       
 24  lnsize     float64       
dtypes: datetime64[ns](1), float64(19), int64(1)

In [ ]:
CRSPstock.drop(CRSPstock[CRSPstock.VOL==0].index,axis=0, inplace=True)

In [ ]:
CRSPstock.drop(CRSPstock[CRSPstock.SHRCD.isin([10,11])==False].index,axis=0,inplace=True)

In [ ]:
# take weekly median
CRSPstock = CRSPstock[['PERMNO','yearMonth','yearWeek','BAS','Amihud','OCAmihud','turnover','dvol','size','RET','PRC']].groupby(['PERMNO','yearWeek']).median().reset_index()

In [ ]:
CRSPstock.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2317262 entries, 0 to 2317261
Data columns (total 11 columns):
 #   Column     Dtype  
---  ------     -----  
 0   PERMNO     object 
 1   yearWeek   object 
 2   yearMonth  float64
 3   BAS        float64
 4   Amihud     float64
 5   OCAmihud   float64
 6   turnover   float64
 7   dvol       float64
 8   size       float64
 9   RET        float64
 10  PRC        float64
dtypes: float64(9), object(2)
memory usage: 194.5+ MB


In [ ]:
CRSPstock[['BAS','Amihud','OCAmihud']].describe(percentiles = [0.0001,0.001,0.01,0.99,0.999,0.9999])

,BAS,Amihud,OCAmihud
count,2316255.000000,2316265.000000,2313800.000000
mean,0.659644,100.291102,21.974766
std,1.553836,9216.453429,237.018286
min,0.000000,0.000000,0.000000
0.01%,0.001468,0.000000,0.000000
0.1%,0.003913,0.000000,0.000000
1%,0.008803,0.000744,0.000000
50%,0.132163,0.369797,0.320171
99%,6.896552,867.541177,379.322002
99.9%,16.099622,9136.786011,2080.552461


In [ ]:
CRSPstock['BASdiff'] =  CRSPstock.groupby(['PERMNO']).BAS.diff()

CRSPstock['lnAdiff'] = np.log(CRSPstock['Amihud']+1)
CRSPstock['lnAdiff'] =  CRSPstock.groupby(['PERMNO']).lnAdiff.diff()

CRSPstock['lnOCAdiff'] = np.log(CRSPstock['OCAmihud']+1)
CRSPstock['lnOCAdiff'] =  CRSPstock.groupby(['PERMNO']).lnOCAdiff.diff()

In [ ]:
CRSPstock[['BASdiff','lnAdiff','lnOCAdiff']].describe(percentiles = [0.0001,0.001,0.01,0.99,0.999,0.9999])

,BASdiff,lnAdiff,lnOCAdiff
count,2309581.000000,2309591.000000,2305348.000000
mean,-0.001326,-0.001093,-0.000878
std,0.988441,0.688892,0.758555
min,-163.126689,-14.695248,-10.858448
0.01%,-17.954418,-7.851335,-7.793244
0.1%,-7.310721,-4.902112,-5.699227
1%,-2.513089,-2.175318,-2.443357
50%,-0.000031,0.000016,0.000000
99%,2.486418,2.109449,2.360974
99.9%,7.342297,4.814688,5.675364


In [ ]:
CRSPstock.dropna(axis=0, inplace=True)

In [ ]:
CRSPstock['year'] = CRSPstock.yearWeek.str[0:4].astype(int)
CRSPstock.drop(CRSPstock[CRSPstock.year<=2010].index,inplace=True,axis=0)

In [ ]:
CRSPstock.to_pickle('/content/drive/My Drive/Projects/Illiquidity_Contagion/Data/Raw/CRSPstock_week.pkl')

In [ ]:
with open('/content/drive/My Drive/Projects/Illiquidity_Contagion/Data/Raw/CRSPstock_week.pkl', 'rb') as f:
    CRSPstock = pickle.load(f)

In [ ]:
CRSPstock[['BASdiff','lnAdiff']].describe(percentiles = [0.0001,0.001,0.01,0.99,0.999,0.9999])

,BASdiff,lnAdiff
count,3234271.000000,3234271.000000
mean,-0.001176,-0.000821
std,0.918490,0.635052
min,-163.126689,-14.695248
0.01%,-17.026692,-7.685452
0.1%,-6.721153,-4.597968
1%,-2.233963,-2.002310
50%,-0.000025,0.000016
99%,2.214003,1.939320
99.9%,6.795144,4.529739


In [ ]:
CRSPstock[['BASdiff','lnAdiff','lnOCAdiff']].corr()

,BASdiff,lnAdiff,lnOCAdiff
BASdiff,1.000000,0.132772,0.061501
lnAdiff,0.132772,1.000000,0.307609
lnOCAdiff,0.061501,0.307609,1.000000


# RQ1: Koch et al (2016) and Agarwal et al (2019)
To show that the liquidity of a portfolio of large cap ETFs have a positive influence on stock liquidity, after controlling for the liqudity of a high ETF ownership stock portfolio, and the liquidity of a market portfolio.


Steps:
1. create a market portfolio
2. measure ETF ownership, and create a high ETF ownership stock portfolio
3. measure ETF market cap, and create a large ETF portfolio
4. regression

## market portfolio

In [ ]:
CRSPstock.describe()

,yearMonth,BAS,Amihud,OCAmihud,turnover,dvol,size,RET,PRC,BASdiff,lnAdiff,lnOCAdiff,year
count,2962600.000000,2962600.000000,2962600.000000,2962600.000000,2962600.000000,2962600.000000,2962600.000000,2962600.000000,2962600.000000,2962600.000000,2962600.000000,2962600.000000,2962600.000000
mean,201609.199527,0.556733,50.252090,17.175950,1.172503,53.800689,6144.643896,-0.000256,82.451107,-0.000539,-0.000586,-0.000484,2016.026759
std,316.273280,1.318439,1836.326345,194.014757,20.838506,452.788084,31558.245597,0.015994,3544.593352,0.849403,0.612612,0.681939,3.162462
min,201101.000000,0.000000,0.000000,0.000000,0.000002,0.000002,0.031500,-0.818667,0.001550,-163.126689,-14.149227,-10.858448,2011.000000
25%,201310.000000,0.043927,0.040128,0.033931,0.231251,0.501225,171.122840,-0.005996,7.290000,-0.016670,-0.083323,-0.078596,2013.000000
50%,201607.000000,0.121286,0.282574,0.247787,0.490946,3.840703,690.108685,0.000000,17.890000,-0.000018,0.000020,0.000012,2016.000000
75%,201904.000000,0.462211,2.683407,2.341718,0.935051,24.456916,2951.871840,0.005807,40.930000,0.017595,0.086722,0.081993,2019.000000
max,202112.000000,163.544028,746699.670380,58138.778418,16511.741043,84851.522710,2930481.081221,1.463636,450662.000000,163.279612,13.150035,10.839084,2021.000000


In [ ]:
df = CRSPstock[['RET','size','yearWeek']].groupby('yearWeek').apply(wavg, "RET", "size").reset_index()

In [ ]:
df['MKT_BASdiff'] = CRSPstock[['BASdiff','size','yearWeek']].groupby('yearWeek').apply(wavg, "BASdiff", "size").values
df['MKT_lnAdiff'] = CRSPstock[['lnAdiff','size','yearWeek']].groupby('yearWeek').apply(wavg, "lnAdiff", "size").values
df['MKT_lnOCAdiff'] = CRSPstock[['lnOCAdiff','size','yearWeek']].groupby('yearWeek').apply(wavg, "lnOCAdiff", "size").values

In [ ]:
df.columns = ['yearWeek','MKT_RET','MKT_BASdiff','MKT_lnAdiff','MKT_lnOCAdiff']

In [ ]:
df.corr()

,MKT_RET,MKT_BASdiff,MKT_lnAdiff,MKT_lnOCAdiff
MKT_RET,1.000000,-0.328052,-0.320266,-0.287139
MKT_BASdiff,-0.328052,1.000000,0.381264,0.412968
MKT_lnAdiff,-0.320266,0.381264,1.000000,0.694014
MKT_lnOCAdiff,-0.287139,0.412968,0.694014,1.000000


In [ ]:
df.to_pickle('/content/drive/My Drive/Projects/Illiquidity_Contagion/Data/Prepared/RQ1_portMKT.pkl')

## High ETF ownership portfolio

In [ ]:
with open('/content/drive/My Drive/Projects/Illiquidity_Contagion/Data/Prepared/RQ1_ETFownership.pkl', 'rb') as f:
    df = pickle.load(f)

In [ ]:
df['yearMonth'] = df['time'].astype(str).str.replace('-','').astype(int)

In [ ]:
df = CRSPstock[['PERMNO','yearMonth','yearWeek','BASdiff','lnAdiff','lnOCAdiff','size']].merge(df[['PERMNO','yearMonth','ETFownership','HighETFownership']],on=['PERMNO','yearMonth'],how='left')

/usr/local/lib/python3.7/dist-packages/pandas/core/reshape/merge.py:1208: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  UserWarning,


In [ ]:
df

,PERMNO,yearMonth,yearWeek,BASdiff,lnAdiff,size,ETFownership,HighETFownership
0,10001,201101.000000,2011-01,0.193220,0.548159,81.786960,0.423845,0.000000
1,10001,201101.000000,2011-02,-0.198586,-0.319033,83.118740,0.423845,0.000000
2,10001,201101.000000,2011-03,0.230749,0.500389,83.823408,0.423845,0.000000
3,10001,201101.000000,2011-04,-0.144807,0.139317,84.607200,0.423845,0.000000
4,10001,201102.000000,2011-05,-0.180579,-1.174227,84.442686,0.208846,0.000000
...,...,...,...,...,...,...,...,...
2966671,93436,202112.000000,2021-48,-0.003323,0.000100,1099670.175000,NaN,NaN
2966672,93436,202112.000000,2021-49,-0.015912,-0.000064,1021367.663078,NaN,NaN
2966673,93436,202112.000000,2021-50,0.023671,-0.000035,962598.055193,NaN,NaN
2966674,93436,202112.000000,2021-51,-0.020885,0.000125,977852.845564,NaN,NaN


In [ ]:
df.dropna(axis=0,inplace=True)
df = df[df.HighETFownership==1]

In [ ]:
ownership = df[['BASdiff','size','yearWeek']].groupby('yearWeek').apply(wavg, "BASdiff", "size").reset_index()
ownership['ownership_lnAdiff'] = df[['lnAdiff','size','yearWeek']].groupby('yearWeek').apply(wavg, "lnAdiff", "size").values
ownership['ownership_lnOCAdiff'] = df[['lnOCAdiff','size','yearWeek']].groupby('yearWeek').apply(wavg, "lnOCAdiff", "size").values

In [ ]:
ownership.columns = ['yearWeek','ownership_BASdiff','ownership_lnAdiff','ownership_lnOCAdiff']

In [ ]:
ownership.corr()

,ownership_BASdiff,ownership_lnAdiff,ownership_lnOCAdiff
ownership_BASdiff,1.000000,0.188334,0.254511
ownership_lnAdiff,0.188334,1.000000,0.129161
ownership_lnOCAdiff,0.254511,0.129161,1.000000


In [ ]:
ownership.to_pickle('/content/drive/My Drive/Projects/Illiquidity_Contagion/Data/Prepared/RQ1_Highownership.pkl')

## ETF portfolio

### ETF marekt port

In [ ]:
df = CRSPetf[['yearWeek','size','BASdiff']].groupby(['yearWeek']).apply(wavg, "BASdiff", "size").reset_index()
df['ETF_lnAdiff'] = CRSPetf[['yearWeek','size','lnAdiff']].groupby(['yearWeek']).apply(wavg, "lnAdiff", "size").values
df['ETF_lnOCAdiff'] = CRSPetf[['yearWeek','size','lnOCAdiff']].groupby(['yearWeek']).apply(wavg, "lnOCAdiff", "size").values

In [ ]:
df.columns = ['yearWeek','ETF_BASdiff','ETF_lnAdiff','ETF_lnOCAdiff']

In [ ]:
df.corr()

,ETF_BASdiff,ETF_lnAdiff,ETF_lnOCAdiff
ETF_BASdiff,1.000000,0.188711,0.221565
ETF_lnAdiff,0.188711,1.000000,0.561001
ETF_lnOCAdiff,0.221565,0.561001,1.000000


In [ ]:
df.to_pickle('/content/drive/My Drive/Projects/Illiquidity_Contagion/Data/Prepared/RQ1_ETFport.pkl')

### highly liquid ETF portfolio

In [ ]:
CRSPetf['BAS_rank'] = CRSPetf[['BAS','yearWeek']].groupby('yearWeek').rank().values

In [ ]:
CRSPetf.loc[CRSPetf.yearWeek=='2020-21',('TICKER','size','BAS','BAS_rank')].sort_values(by='BAS',ascending = True).head(50)

,TICKER,size,BAS,BAS_rank
282008,SPY,266645.190000,0.003383,1.000000
322917,VOO,134549.739797,0.003730,2.000000
35764,DIA,19489.625860,0.004072,3.000000
224675,QQQ,107920.296000,0.004396,4.000000
113737,IJH,41220.255000,0.006053,5.000000
124225,IVV,185913.126280,0.006754,6.000000
128614,IWM,36273.800000,0.007464,7.000000
346277,XLY,12285.881940,0.008349,8.000000
345599,XLV,24833.947950,0.009998,9.000000
343145,XLK,28510.893840,0.010403,10.000000


In [ ]:
df = CRSPetf.loc[CRSPetf.BAS_rank<=10,['yearWeek','size','BASdiff']].groupby(['yearWeek']).apply(wavg, "BASdiff", "size").reset_index()

In [ ]:
df.columns = ['yearWeek','HliqETF_BASdiff']

In [ ]:
df.to_pickle('/content/drive/My Drive/Projects/Illiquidity_Contagion/Data/Prepared/RQ1_HliqETFport.pkl')

## Test hypothesis

In [ ]:
with open('/content/drive/My Drive/Projects/Illiquidity_Contagion/Data/Prepared/RQ1_portMKT.pkl', 'rb') as f:
    mkt = pickle.load(f)

with open('/content/drive/My Drive/Projects/Illiquidity_Contagion/Data/Prepared/RQ1_Highownership.pkl', 'rb') as f:
    highOwnership = pickle.load(f)

with open('/content/drive/My Drive/Projects/Illiquidity_Contagion/Data/Prepared/RQ1_ETFport.pkl', 'rb') as f:
    ETFs = pickle.load(f)

with open('/content/drive/My Drive/Projects/Illiquidity_Contagion/Data/Prepared/RQ1_HliqETFport.pkl', 'rb') as f:
    HliqETFs = pickle.load(f)

In [ ]:
df = mkt.merge(highOwnership,on='yearWeek')
df = df.merge(ETFs,on='yearWeek')
df = df.merge(HliqETFs,on='yearWeek')
df.dropna(axis=0,inplace=True)

In [ ]:
df.describe()

,MKT_RET,MKT_BASdiff,MKT_lnAdiff,MKT_lnOCAdiff,ownership_BASdiff,ownership_lnAdiff,ownership_lnOCAdiff,ETF_BASdiff,ETF_lnAdiff,ETF_lnOCAdiff,HliqETF_BASdiff
count,567.000000,567.000000,567.000000,567.000000,567.000000,567.000000,567.000000,567.000000,567.000000,567.000000,567.000000
mean,0.000508,-0.000084,-0.000106,-0.000089,-0.000022,0.000030,-0.000475,-0.000011,-0.000001,0.000032,-0.001028
std,0.004894,0.003123,0.005187,0.004497,0.003620,0.010453,0.011963,0.003150,0.011169,0.009068,0.001892
min,-0.044130,-0.019919,-0.023900,-0.025605,-0.025553,-0.044384,-0.202870,-0.019917,-0.038138,-0.043836,-0.027119
25%,-0.001424,-0.001289,-0.002976,-0.002367,-0.001221,-0.005358,-0.004238,-0.001066,-0.006672,-0.004891,-0.001476
50%,0.000897,-0.000227,-0.000275,-0.000158,-0.000108,-0.000113,-0.000139,-0.000057,-0.000439,-0.000099,-0.000698
75%,0.002971,0.000894,0.002317,0.001962,0.001016,0.004780,0.003578,0.000837,0.006598,0.005544,-0.000224
max,0.027143,0.027861,0.031170,0.030514,0.030896,0.105542,0.047530,0.038785,0.046925,0.058230,0.002909


In [ ]:
df[['MKT_BASdiff','ownership_BASdiff','ETF_BASdiff','HliqETF_BASdiff']].corr()

,MKT_BASdiff,ownership_BASdiff,ETF_BASdiff,HliqETF_BASdiff
MKT_BASdiff,1.000000,0.848049,0.567626,-0.128733
ownership_BASdiff,0.848049,1.000000,0.425713,-0.150933
ETF_BASdiff,0.567626,0.425713,1.000000,0.102011
HliqETF_BASdiff,-0.128733,-0.150933,0.102011,1.000000


### Main specification


In [ ]:
with open('/content/drive/My Drive/Projects/Illiquidity_Contagion/Data/Raw/CRSPstock_week.pkl', 'rb') as f:
    CRSPstock = pickle.load(f)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 567 entries, 0 to 566
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   yearWeek             567 non-null    object 
 1   MKT_RET              567 non-null    float64
 2   MKT_BASdiff          567 non-null    float64
 3   MKT_lnAdiff          567 non-null    float64
 4   MKT_lnOCAdiff        567 non-null    float64
 5   ownership_BASdiff    567 non-null    float64
 6   ownership_lnAdiff    567 non-null    float64
 7   ownership_lnOCAdiff  567 non-null    float64
 8   ETF_BASdiff          567 non-null    float64
 9   ETF_lnAdiff          567 non-null    float64
 10  ETF_lnOCAdiff        567 non-null    float64
 11  HliqETF_BASdiff      567 non-null    float64
dtypes: float64(11), object(1)
memory usage: 57.6+ KB


In [ ]:
df['lead_MKT'] = df.MKT_BASdiff.shift(-1)
df['lag_MKT'] = df.MKT_BASdiff.shift(1)
#df['lead_ownership'] = df.ownership_BASdiff.shift(-1)
#df['lag_ownership'] = df.ownership_BASdiff.shift(1)
df['lead_ETF'] = df.ETF_BASdiff.shift(-1)
df['lag_ETF'] = df.ETF_BASdiff.shift(1)

df['lead_HliqETF'] = df.HliqETF_BASdiff.shift(-1)
df['lag_HliqETF'] = df.HliqETF_BASdiff.shift(1)

df['lead_RET'] = df.MKT_RET.shift(-1)
df['lag_RET'] = df.MKT_RET.shift(1)
df.dropna(axis=0,inplace=True)

In [ ]:
save = pd.DataFrame()
for stock in CRSPstock.PERMNO.unique():
    reg_data = CRSPstock.loc[CRSPstock.PERMNO == stock,('yearWeek','BASdiff','RET')]
    reg_data = reg_data.merge(df,on='yearWeek')
    reg_data.dropna(axis=0,inplace=True)

    if len(reg_data) < 52*8:
        continue

    y = reg_data['BASdiff']
    X = reg_data[['lead_ETF','ETF_BASdiff','lag_ETF',
                  'lead_HliqETF','HliqETF_BASdiff','lag_HliqETF',
                  'lead_MKT','MKT_BASdiff','lag_MKT',
                  'lead_RET','MKT_RET','lag_RET',
                  'RET']]
    X.RET = np.square(X.RET)
    X = sm.add_constant(X)
    model = sm.OLS(y, X).fit()
    
    save[stock] = model.params[1:].values

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


In [ ]:
save = save.transpose()
save.columns = ['lead_ETF','ETF','lag_ETF',
                  'lead_HliqETF','HliqETF','lag_HliqETF',
                  'lead_MKT','MKT_LIQ','lag_MKT',
                  'lead_RET','MKT_RET','lag_RET',
                  'squared_RET']

In [ ]:
for var in save.columns[1:]:
    save[var] = save[var].astype(float)

In [ ]:
save.describe()

,lead_ETF,ETF,lag_ETF,lead_HliqETF,HliqETF,lag_HliqETF,lead_MKT,MKT_LIQ,lag_MKT,lead_RET,MKT_RET,lag_RET,squared_RET
count,2526.000000,2526.000000,2526.000000,2526.000000,2526.000000,2526.000000,2526.000000,2526.000000,2526.000000,2526.000000,2526.000000,2526.000000,2526.000000
mean,-0.257364,5.319855,-2.061517,2.141004,-1.934079,2.944351,-1.024655,5.220191,0.499084,0.082491,-1.398103,-0.988050,20.178883
std,17.180406,25.764128,23.144176,28.520770,30.383084,29.796253,17.535653,18.894260,19.080610,8.452299,10.158143,10.039049,136.861536
min,-143.221101,-163.469138,-233.413831,-268.273804,-234.952788,-398.327447,-181.144248,-165.874812,-348.676569,-70.289363,-104.286996,-166.763473,-780.597806
25%,-0.963183,-0.503013,-2.367164,-1.019677,-3.252313,-1.245328,-0.961635,0.340170,-0.974267,-0.344951,-0.735396,-0.853720,-6.077136
50%,-0.033437,0.267261,-0.274240,0.130160,-0.151014,0.086519,-0.010206,1.355327,0.104907,0.030362,-0.045278,-0.063067,-0.355060
75%,0.881966,3.520282,0.969915,2.343264,1.190293,2.600199,0.828990,4.929686,1.265610,0.584998,0.483234,0.337423,6.523436
max,140.392322,327.181506,251.228227,209.643386,315.230757,391.728039,144.143837,267.225781,233.080244,64.124829,85.441842,63.379551,1940.677032


In [ ]:
for var in ['ETF','HliqETF','MKT_LIQ']:
    ic([var, ttest_1samp(save[var],popmean=0)[0]])

ic| [var, ttest_1samp(save[var],popmean=0)[0]]: ['ETF', 10.377697388874214]
ic| [var, ttest_1samp(save[var],popmean=0)[0]]: ['HliqETF', -3.1993295152987695]
ic| [var, ttest_1samp(save[var],popmean=0)[0]]: ['MKT_LIQ', 13.885873270347117]


In [ ]:
save.to_pickle('/content/drive/My Drive/Projects/Illiquidity_Contagion/Data/Prepared/RQ1.pkl')

Top25%

In [ ]:
with open('/content/drive/My Drive/Projects/Illiquidity_Contagion/Data/Raw/CRSPstock_week.pkl', 'rb') as f:
    CRSPstock = pickle.load(f)

In [ ]:
# get the 75% percentile in market cap
temp = CRSPstock[["yearWeek","size"]].groupby(["yearWeek"]).quantile(0.75).reset_index()
temp.columns = ["yearWeek","cutoff_size"]

In [ ]:
# drop observations below median size
subset = CRSPstock.merge(temp,how='left',on='yearWeek')
todrop = subset.loc[subset['size']<=subset.cutoff_size,'PERMNO'].unique()
CRSPstock.drop(CRSPstock[CRSPstock.PERMNO.isin(todrop)].index,axis=0,inplace=True)

In [ ]:
save = pd.DataFrame()
for stock in CRSPstock.PERMNO.unique():
    reg_data = CRSPstock.loc[CRSPstock.PERMNO == stock,('yearWeek','BASdiff','RET')]
    reg_data = reg_data.merge(df,on='yearWeek')
    reg_data.dropna(axis=0,inplace=True)

    if len(reg_data) < 52*8:
        continue

    y = reg_data['BASdiff']
    X = reg_data[['lead_ETF','ETF_BASdiff','lag_ETF',
                  'lead_HliqETF','HliqETF_BASdiff','lag_HliqETF',
                  'lead_MKT','MKT_BASdiff','lag_MKT',
                  'lead_RET','MKT_RET','lag_RET',
                  'RET']]
    X.RET = np.square(X.RET)
    X = sm.add_constant(X)
    model = sm.OLS(y, X).fit()
    
    save[stock] = model.params[1:].values

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


In [ ]:
save = save.transpose()
save.columns = ['lead_ETF','ETF','lag_ETF',
                  'lead_HliqETF','HliqETF','lag_HliqETF',
                  'lead_MKT','MKT_LIQ','lag_MKT',
                  'lead_RET','MKT_RET','lag_RET',
                  'squared_RET']

In [ ]:
for var in save.columns[1:]:
    save[var] = save[var].astype(float)

In [ ]:
save.describe()

,lead_ETF,ETF,lag_ETF,lead_HliqETF,HliqETF,lag_HliqETF,lead_MKT,MKT_LIQ,lag_MKT,lead_RET,MKT_RET,lag_RET,squared_RET
count,474.000000,474.000000,474.000000,474.000000,474.000000,474.000000,474.000000,474.000000,474.000000,474.000000,474.000000,474.000000,474.000000
mean,-0.017493,0.010431,0.046098,0.038821,-0.059016,-0.085404,0.031424,0.693783,0.079519,-0.004651,0.003377,0.010389,-1.107470
std,0.475387,0.647240,0.934775,0.775987,1.018521,0.939557,0.574673,0.674228,0.633742,0.264472,0.257664,0.268556,9.648600
min,-5.061586,-2.130851,-5.807936,-7.330586,-5.506375,-6.489749,-4.201693,-0.727210,-2.127945,-3.536879,-1.267456,-1.533084,-131.822648
25%,-0.199055,-0.355043,-0.410751,-0.256668,-0.396029,-0.462565,-0.152923,0.224302,-0.217164,-0.093132,-0.118545,-0.100689,-3.365681
50%,-0.019727,-0.021480,-0.031197,-0.001796,-0.009585,-0.044239,0.015292,0.573461,0.026018,0.000862,-0.010960,-0.001340,-0.619430
75%,0.146984,0.225989,0.441577,0.305997,0.480904,0.273185,0.215773,1.007659,0.320641,0.076079,0.126185,0.117393,1.761120
max,4.351348,3.478013,4.245427,4.381712,7.156174,4.532384,5.585546,5.190257,4.286452,1.581413,1.193803,1.466892,68.407870


In [ ]:
for var in ['ETF','HliqETF','MKT_LIQ']:
    ic([var, ttest_1samp(save[var],popmean=0)[0]])

ic| [var, ttest_1samp(save[var],popmean=0)[0]]: ['ETF', 0.35086320901103335]
ic| [var, ttest_1samp(save[var],popmean=0)[0]]: ['HliqETF', -1.261515388855357]
ic| [var, ttest_1samp(save[var],popmean=0)[0]]: ['MKT_LIQ', 22.402991703851335]


### Test specification 1: BAS

In [ ]:
with open('/content/drive/My Drive/Projects/Illiquidity_Contagion/Data/Raw/CRSPstock_week.pkl', 'rb') as f:
    CRSPstock = pickle.load(f)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 567 entries, 0 to 566
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   yearWeek           567 non-null    object 
 1   MKT_RET            567 non-null    float64
 2   MKT_BASdiff        567 non-null    float64
 3   MKT_lnAdiff        567 non-null    float64
 4   ownership_BASdiff  567 non-null    float64
 5   ownership_lnAdiff  567 non-null    float64
 6   ETF_BASdiff        567 non-null    float64
 7   ETF_lnAdiff        567 non-null    float64
dtypes: float64(7), object(1)
memory usage: 39.9+ KB


In [ ]:
df['lead_MKT'] = df.MKT_BASdiff.shift(-1)
df['lag_MKT'] = df.MKT_BASdiff.shift(1)
df['lead_ownership'] = df.ownership_BASdiff.shift(-1)
df['lag_ownership'] = df.ownership_BASdiff.shift(1)
df['lead_ETF'] = df.ETF_BASdiff.shift(-1)
df['lag_ETF'] = df.ETF_BASdiff.shift(1)
df['lead_RET'] = df.MKT_RET.shift(-1)
df['lag_RET'] = df.MKT_RET.shift(1)
df.dropna(axis=0,inplace=True)

In [ ]:
save = pd.DataFrame()
for stock in CRSPstock.PERMNO.unique():
    reg_data = CRSPstock.loc[CRSPstock.PERMNO == stock,('yearWeek','BASdiff','RET')]
    reg_data = reg_data.merge(df,on='yearWeek')
    reg_data.dropna(axis=0,inplace=True)

    if len(reg_data) < 52*8:
        continue

    y = reg_data['BASdiff']
    X = reg_data[['lead_ETF','ETF_BASdiff','lag_ETF',
                  'lead_ownership','ownership_BASdiff','lag_ownership',
                  'lead_MKT','MKT_BASdiff','lag_MKT',
                  'lead_RET','MKT_RET','lag_RET',
                  'RET']]
    X.RET = np.square(X.RET)
    X = sm.add_constant(X)
    model = sm.OLS(y, X).fit()
    
    save[stock] = model.params[1:].values

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


In [ ]:
save = save.transpose()
save.columns = ['lead_ETF','ETF','lag_ETF',
                'lead_ownership','ownership','lag_ownership',
                'lead_MKT_LIQ','MKT_LIQ','lag_MKT_LIQ','lead_MKT_RET','MKT_RET','lag_MKT_RET','squared_RET']

In [ ]:
save.describe()

,lead_ETF,ETF,lag_ETF,lead_ownership,ownership,lag_ownership,lead_MKT_LIQ,MKT_LIQ,lag_MKT_LIQ,lead_MKT_RET,MKT_RET,lag_MKT_RET,squared_RET
count,2526.000000,2526.000000,2526.000000,2526.000000,2526.000000,2526.000000,2526.000000,2526.000000,2526.000000,2526.000000,2526.000000,2526.000000,2526.000000
mean,0.150101,5.091591,-0.969705,0.480749,-1.375211,1.033178,-1.382869,6.375003,-0.899126,0.222515,-1.098862,-0.932843,16.970503
std,17.147498,23.249238,18.992867,21.641088,22.396512,19.756493,29.859827,30.343193,25.623467,8.654906,9.892126,9.912033,121.706059
min,-150.365055,-112.360430,-191.766935,-202.939310,-424.403402,-281.696505,-413.856906,-295.453892,-239.297544,-76.655735,-110.921392,-166.665335,-619.614779
25%,-0.886407,-0.462456,-1.638639,-0.962388,-0.250688,-0.945461,-1.177011,-0.515984,-1.483375,-0.316368,-0.746014,-0.970693,-5.255832
50%,-0.029038,0.226451,-0.193251,0.022633,0.609347,0.076610,0.039812,0.348908,-0.016268,0.046986,-0.078257,-0.077210,-0.174827
75%,0.920549,3.340825,0.871231,1.173553,2.738323,1.356440,1.239734,3.276938,1.019982,0.760778,0.398661,0.293823,5.504232
max,139.532663,390.934499,178.910015,239.817455,176.818891,181.389002,270.302778,472.816649,299.750741,74.811779,94.113092,60.356313,1910.181204


In [ ]:
for var in ['ETF','ownership','MKT_LIQ']:
    ic([var, ttest_1samp(save[var],popmean=0)[0]])

ic| [var, ttest_1samp(save[var],popmean=0)[0]]: ['ETF', 11.006809902099148]
ic| [var, ttest_1samp(save[var],popmean=0)[0]]: ['ownership', -3.08606947146697]
ic| [var, ttest_1samp(save[var],popmean=0)[0]]: ['MKT_LIQ', 10.559316442416366]


In [ ]:
save.to_pickle('/content/drive/My Drive/Projects/Illiquidity_Contagion/Data/Prepared/RQ1.pkl')

Top50%

In [ ]:
with open('/content/drive/My Drive/Projects/Illiquidity_Contagion/Data/Raw/CRSPstock_week.pkl', 'rb') as f:
    CRSPstock = pickle.load(f)

In [ ]:
CRSPstock[['BASdiff','lnAdiff']].corr()

,BASdiff,lnAdiff
BASdiff,1.000000,0.063753
lnAdiff,0.063753,1.000000


In [ ]:
# get the size median over time
temp = CRSPstock[["yearWeek","size"]].groupby(["yearWeek"]).quantile(0.50).reset_index()
temp.columns = ["yearWeek","cutoff_size"]

In [ ]:
# drop observations below median size
subset = CRSPstock.merge(temp,how='left',on='yearWeek')
todrop = subset.loc[subset['size']<=subset.cutoff_size,'PERMNO'].unique()
CRSPstock.drop(CRSPstock[CRSPstock.PERMNO.isin(todrop)].index,axis=0,inplace=True)

In [ ]:
save = pd.DataFrame()
for stock in CRSPstock.PERMNO.unique():
    reg_data = CRSPstock.loc[CRSPstock.PERMNO == stock,('yearWeek','BASdiff','RET')]
    reg_data = reg_data.merge(df,on='yearWeek')
    reg_data.dropna(axis=0,inplace=True)

    if len(reg_data) < 52*5:
        continue

    y = reg_data['BASdiff']
    X = reg_data[['lead_ETF','ETF_BASdiff','lag_ETF',
                  'lead_ownership','ownership_BASdiff','lag_ownership',
                  'lead_MKT','MKT_BASdiff','lag_MKT',
                  'lead_RET','MKT_RET','lag_RET',
                  'RET']]
    X.RET = np.square(X.RET)
    X = sm.add_constant(X)
    model = sm.OLS(y, X).fit()
    
    save[stock] = [stock] + list(model.params[1:].values)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


In [ ]:
save = save.transpose()
save.columns = ['PERMNO','lead_ETF','ETF','lag_ETF',
                'lead_ownership','ownership','lag_ownership',
                'lead_MKT_LIQ','MKT_LIQ','lag_MKT_LIQ','lead_MKT_RET','MKT_RET','lag_MKT_RET','squared_RET']

In [ ]:
save.describe()

,PERMNO,lead_ETF,ETF,lag_ETF,lead_ownership,ownership,lag_ownership,lead_MKT_LIQ,MKT_LIQ,lag_MKT_LIQ,lead_MKT_RET,MKT_RET,lag_MKT_RET,squared_RET
count,1686,1686.000000,1686.000000,1686.000000,1686.000000,1686.000000,1686.000000,1686.000000,1686.000000,1686.000000,1686.000000,1686.000000,1686.000000,1686.000000
unique,1686,1686.000000,1686.000000,1686.000000,1686.000000,1686.000000,1686.000000,1686.000000,1686.000000,1686.000000,1686.000000,1686.000000,1686.000000,1686.000000
top,10026,0.057918,6.104338,2.349678,-2.492514,-1.506220,-1.372322,1.942334,2.941378,-0.697247,0.551655,1.806064,0.160283,20.075122
freq,1,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
for var in ['ETF','ownership','MKT_LIQ']:
    ic([var, ttest_1samp(save[var],popmean=0)[0]])

ic| [var, ttest_1samp(save[var],popmean=0)[0]]: ['ETF', 7.450657240802882]
ic| [var, ttest_1samp(save[var],popmean=0)[0]]: ['ownership', 18.056282917693085]
ic| [var, ttest_1samp(save[var],popmean=0)[0]]: ['MKT_LIQ', 7.248841194439078]


Top25%

In [ ]:
with open('/content/drive/My Drive/Projects/Illiquidity_Contagion/Data/Raw/CRSPstock_week.pkl', 'rb') as f:
    CRSPstock = pickle.load(f)

In [ ]:
# get the 75% percentile in market cap
temp = CRSPstock[["yearWeek","size"]].groupby(["yearWeek"]).quantile(0.75).reset_index()
temp.columns = ["yearWeek","cutoff_size"]

In [ ]:
# drop observations below median size
subset = CRSPstock.merge(temp,how='left',on='yearWeek')
todrop = subset.loc[subset['size']<=subset.cutoff_size,'PERMNO'].unique()
CRSPstock.drop(CRSPstock[CRSPstock.PERMNO.isin(todrop)].index,axis=0,inplace=True)

In [ ]:
save = pd.DataFrame()
for stock in CRSPstock.PERMNO.unique():
    reg_data = CRSPstock.loc[CRSPstock.PERMNO == stock,('yearWeek','BASdiff','RET')]
    reg_data = reg_data.merge(df,on='yearWeek')
    reg_data.dropna(axis=0,inplace=True)

    if len(reg_data) < 52*5:
        continue

    y = reg_data['BASdiff']
    X = reg_data[['lead_ETF','ETF_BASdiff','lag_ETF',
                  'lead_ownership','ownership_BASdiff','lag_ownership',
                  'lead_MKT','MKT_BASdiff','lag_MKT',
                  'lead_RET','MKT_RET','lag_RET',
                  'RET']]
    X.RET = np.square(X.RET)
    X = sm.add_constant(X)
    model = sm.OLS(y, X).fit()
    
    save[stock] = [stock] + list(model.params[1:].values)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


In [ ]:
save = save.transpose()
save.columns = ['PERMNO','lead_ETF','ETF','lag_ETF',
                'lead_ownership','ownership','lag_ownership',
                'lead_MKT_LIQ','MKT_LIQ','lag_MKT_LIQ','lead_MKT_RET','MKT_RET','lag_MKT_RET','squared_RET']

In [ ]:
for var in save.columns[1:]:
    save[var] = save[var].astype(float)

In [ ]:
save.describe()

,lead_ETF,ETF,lag_ETF,lead_ownership,ownership,lag_ownership,lead_MKT_LIQ,MKT_LIQ,lag_MKT_LIQ,lead_MKT_RET,MKT_RET,lag_MKT_RET,squared_RET
count,556.000000,556.000000,556.000000,556.000000,556.000000,556.000000,556.000000,556.000000,556.000000,556.000000,556.000000,556.000000,556.000000
mean,-0.025264,-0.031567,0.003017,-0.017030,0.373502,-0.014076,0.049820,0.304968,0.081077,0.007263,-0.023445,0.000751,-0.278938
std,0.490616,0.606040,0.722830,0.537039,0.704489,0.558662,0.808398,0.594312,0.637276,0.267334,0.258296,0.257512,7.251645
min,-5.661277,-2.914629,-5.282288,-6.390454,-1.601470,-3.350263,-5.300054,-2.019851,-3.039008,-3.324683,-1.544911,-1.353292,-95.613552
25%,-0.205408,-0.303863,-0.296547,-0.217266,0.009513,-0.261357,-0.173380,0.004037,-0.183236,-0.094744,-0.127702,-0.109604,-2.240645
50%,-0.033467,-0.048165,-0.026740,-0.020437,0.247439,0.013641,0.052691,0.245011,0.024562,0.003537,-0.030626,-0.011597,-0.289489
75%,0.113985,0.178326,0.268023,0.189307,0.571091,0.236773,0.286893,0.576896,0.272295,0.088299,0.088757,0.111547,1.242605
max,4.422463,4.684190,2.962107,2.939131,6.963300,3.024215,11.468191,3.659937,5.189840,1.850841,1.217976,1.487046,49.004062


In [ ]:
for var in ['ETF','ownership','MKT_LIQ']:
    ic([var, ttest_1samp(save[var],popmean=0)[0]])

ic| [var, ttest_1samp(save[var],popmean=0)[0]]: ['ETF', -1.2281970848711636]
ic| [var, ttest_1samp(save[var],popmean=0)[0]]: ['ownership', 12.501343114299285]
ic| [var, ttest_1samp(save[var],popmean=0)[0]]: ['MKT_LIQ', 12.09977115222745]


### Robustness: ln(Amihud+1)

In [ ]:
with open('/content/drive/My Drive/Projects/Illiquidity_Contagion/Data/Raw/CRSPstock_week.pkl', 'rb') as f:
    CRSPstock = pickle.load(f)

In [ ]:
CRSPstock.lnAdiff = winsorization(CRSPstock.lnAdiff, 0.01)

In [ ]:
todrop = CRSPstock.loc[CRSPstock.PRC<2,'PERMNO'].unique()
CRSPstock.drop(CRSPstock[CRSPstock.PERMNO.isin(todrop)].index,axis=0,inplace=True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 565 entries, 1 to 565
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   yearWeek             565 non-null    object 
 1   MKT_RET              565 non-null    float64
 2   MKT_BASdiff          565 non-null    float64
 3   MKT_lnAdiff          565 non-null    float64
 4   MKT_lnOCAdiff        565 non-null    float64
 5   ownership_BASdiff    565 non-null    float64
 6   ownership_lnAdiff    565 non-null    float64
 7   ownership_lnOCAdiff  565 non-null    float64
 8   ETF_BASdiff          565 non-null    float64
 9   ETF_lnAdiff          565 non-null    float64
 10  ETF_lnOCAdiff        565 non-null    float64
 11  lead_MKT             565 non-null    float64
 12  lag_MKT              565 non-null    float64
 13  lead_ownership       565 non-null    float64
 14  lag_ownership        565 non-null    float64
 15  lead_ETF             565 non-null    flo

In [ ]:
df['lead_MKT'] = df.MKT_lnAdiff.shift(-1)
df['lag_MKT'] = df.MKT_lnAdiff.shift(1)
df['lead_ownership'] = df.ownership_lnAdiff.shift(-1)
df['lag_ownership'] = df.ownership_lnAdiff.shift(1)
df['lead_ETF'] = df.ETF_lnAdiff.shift(-1)
df['lag_ETF'] = df.ETF_lnAdiff.shift(1)
df['lead_RET'] = df.MKT_RET.shift(-1)
df['lag_RET'] = df.MKT_RET.shift(1)
df.dropna(axis=0,inplace=True)

In [ ]:
save=pd.DataFrame()
for stock in CRSPstock.PERMNO.unique():
    reg_data = CRSPstock.loc[CRSPstock.PERMNO == stock,('yearWeek','lnAdiff','RET')]
    reg_data = reg_data.merge(df,on='yearWeek')
    reg_data.dropna(axis=0,inplace=True)

    if len(reg_data) < 52*8:
        continue

    y = reg_data['lnAdiff']
    X = reg_data[['lead_ETF','ETF_lnAdiff','lag_ETF',
                  'lead_ownership','ownership_lnAdiff','lag_ownership',
                  'lead_MKT','MKT_lnAdiff','lag_MKT',
                  'lead_RET','MKT_RET','lag_RET',
                  'RET']]
    X.RET = np.square(X.RET)
    X = sm.add_constant(X)
    model = sm.OLS(y, X).fit()
    
    save[stock] = [stock] + list(model.params[1:].values)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


In [ ]:
save = save.transpose()
save.columns = ['PERMNO','lead_ETF','ETF','lag_ETF',
                'lead_ownership','ownership','lag_ownership',
                'lead_MKT_LIQ','MKT_LIQ','lag_MKT_LIQ','lead_MKT_RET','MKT_RET','lag_MKT_RET','squared_RET']

In [ ]:
for var in save.columns[1:]:
    save[var] = save[var].astype(float)

In [ ]:
save.describe(percentiles = [0.0001,0.001,0.01,0.99,0.999,0.9999])

,lead_ETF,ETF,lag_ETF,lead_ownership,ownership,lag_ownership,lead_MKT_LIQ,MKT_LIQ,lag_MKT_LIQ,lead_MKT_RET,MKT_RET,lag_MKT_RET,squared_RET
count,1886.000000,1886.000000,1886.000000,1886.000000,1886.000000,1886.000000,1886.000000,1886.000000,1886.000000,1886.000000,1886.000000,1886.000000,1886.000000
mean,0.263199,-0.546162,0.283998,-0.193101,0.561378,-0.019788,0.193088,7.781801,-0.432450,0.145560,0.571694,1.150233,96.839152
std,2.715219,2.994719,2.869916,3.833425,4.074496,3.765538,8.858891,12.347271,9.033205,4.225478,4.367493,4.851248,166.962023
min,-16.570093,-24.917658,-26.807067,-22.292396,-33.240375,-30.100984,-67.282201,-49.058840,-74.117875,-27.309177,-23.952144,-34.103973,-31.745485
0.01%,-16.486496,-24.241306,-24.799306,-22.032368,-32.971310,-29.408908,-65.252538,-48.326412,-70.659562,-26.910403,-23.572341,-32.965148,-29.893198
0.1%,-13.886025,-20.003138,-16.147116,-20.442652,-28.399029,-26.120210,-54.763239,-38.737346,-53.700943,-23.524169,-21.824552,-27.644278,-18.211782
1%,-8.202254,-12.184731,-8.076139,-12.282970,-14.191008,-11.679653,-31.084905,-8.430626,-31.595483,-13.535622,-13.955198,-12.175694,-0.194868
50%,-0.005644,0.028357,0.014106,-0.037649,0.127587,-0.015391,0.071937,3.080432,0.004379,-0.010803,0.163039,0.174130,33.846726
99%,10.217119,6.838831,11.353536,13.362276,12.714142,13.614125,29.820018,53.931460,27.764678,15.598809,15.182977,18.450775,779.784564
99.9%,14.293415,12.500757,18.967085,25.299706,20.788258,20.861463,48.913793,80.366922,48.749889,28.202881,22.793763,33.272322,1500.676561


In [ ]:
for var in ['ETF','ownership','MKT_LIQ']:
    ic([var, ttest_1samp(save[var],popmean=0)[0]])

ic| [var, ttest_1samp(save[var],popmean=0)[0]]: ['ETF', -7.920195316133041]
ic| [var, ttest_1samp(save[var],popmean=0)[0]]: ['ownership', 5.983461454996071]
ic| [var, ttest_1samp(save[var],popmean=0)[0]]: ['MKT_LIQ', 27.370326704225334]


Amihud top50%

In [ ]:
with open('/content/drive/My Drive/Projects/Illiquidity_Contagion/Data/Raw/CRSPstock_week.pkl', 'rb') as f:
    CRSPstock = pickle.load(f)

In [ ]:
# get the size median over time
temp = CRSPstock[["yearWeek","size"]].groupby(["yearWeek"]).quantile(0.5).reset_index()
temp.columns = ["yearWeek","cutoff_size"]
temp

,yearWeek,cutoff_size
0,2011-01,501.801580
1,2011-02,505.894320
2,2011-03,495.264585
3,2011-04,489.713460
4,2011-05,500.989255
...,...,...
569,2021-48,1047.656860
570,2021-49,1059.578700
571,2021-50,1021.156830
572,2021-51,1032.093245


In [ ]:
# drop observations below median size
subset = CRSPstock.merge(temp,how='left',on='yearWeek')
todrop = subset.loc[subset['size']<=subset.cutoff_size,'PERMNO'].unique()
CRSPstock.drop(CRSPstock[CRSPstock.PERMNO.isin(todrop)].index,axis=0,inplace=True)

In [ ]:
save = pd.DataFrame()
for stock in CRSPstock.PERMNO.unique():
    reg_data = CRSPstock.loc[CRSPstock.PERMNO == stock,('yearWeek','lnAdiff','RET')]
    reg_data = reg_data.merge(df,on='yearWeek')
    reg_data.dropna(axis=0,inplace=True)

    if len(reg_data) < 52*5:
        continue

    y = reg_data['lnAdiff']
    X = reg_data[['lead_ETF','ETF_lnAdiff','lag_ETF',
                  'lead_ownership','ownership_lnAdiff','lag_ownership',
                  'lead_MKT','MKT_lnAdiff','lag_MKT',
                  'lead_RET','MKT_RET','lag_RET',
                  'RET']]
    X.RET = np.square(X.RET)
    X = sm.add_constant(X)
    model = sm.OLS(y, X).fit()
    
    save[stock] = [stock] + list(model.params[1:].values)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


In [ ]:
save = save.transpose()
save.columns = ['PERMNO','lead_ETF','ETF','lag_ETF',
                'lead_ownership','ownership','lag_ownership',
                'lead_MKT_LIQ','MKT_LIQ','lag_MKT_LIQ','lead_MKT_RET','MKT_RET','lag_MKT_RET','squared_RET']

In [ ]:
for var in save.columns[1:]:
    save[var] = save[var].astype(float)

In [ ]:
save.describe()

,lead_ETF,ETF,lag_ETF,lead_ownership,ownership,lag_ownership,lead_MKT_LIQ,MKT_LIQ,lag_MKT_LIQ,lead_MKT_RET,MKT_RET,lag_MKT_RET,squared_RET
count,1686.000000,1686.000000,1686.000000,1686.000000,1686.000000,1686.000000,1686.000000,1686.000000,1686.000000,1686.000000,1686.000000,1686.000000,1686.000000
mean,0.014226,0.110941,0.047031,-0.086258,0.182974,-0.012075,0.178046,1.947235,-0.079728,-0.015732,0.257306,0.266439,42.821137
std,0.761964,0.793729,0.695574,0.860398,1.036243,0.971599,2.443525,3.229785,2.500757,0.930053,1.082127,1.181742,337.936037
min,-9.073836,-6.205904,-6.503552,-10.009332,-14.038837,-7.919808,-24.184887,-8.308423,-55.425383,-7.429628,-9.999715,-9.376386,-11.253960
25%,-0.150702,-0.058437,-0.096190,-0.170413,-0.042549,-0.137369,-0.211004,0.188028,-0.327767,-0.203595,-0.045448,-0.058753,3.855556
50%,-0.014277,0.051479,0.000315,-0.008904,0.023137,-0.005810,0.042232,0.852081,0.007840,-0.011148,0.069067,0.053054,11.866608
75%,0.076036,0.255532,0.123216,0.070083,0.263856,0.086858,0.535768,2.574972,0.346182,0.151923,0.442417,0.443765,32.591682
max,12.155636,12.157514,11.735721,8.001545,11.776877,23.374083,22.782323,33.910559,31.362159,17.536097,11.633612,14.778279,13319.756215


In [ ]:
for var in ['ETF','ownership','MKT_LIQ']:
    ic([var, ttest_1samp(save[var],popmean=0)[0]])

ic| [var, ttest_1samp(save[var],popmean=0)[0]]: ['ETF', 5.739171049422911]
ic| [var, ttest_1samp(save[var],popmean=0)[0]]: ['ownership', 7.250306719463014]
ic| [var, ttest_1samp(save[var],popmean=0)[0]]: ['MKT_LIQ', 24.755604452498638]


Amihud top75%

In [ ]:
with open('/content/drive/My Drive/Projects/Illiquidity_Contagion/Data/Raw/CRSPstock_week.pkl', 'rb') as f:
    CRSPstock = pickle.load(f)

In [ ]:
# get the size median over time
temp = CRSPstock[["yearWeek","size"]].groupby(["yearWeek"]).quantile(0.25).reset_index()
temp.columns = ["yearWeek","cutoff_size"]
temp

,yearWeek,cutoff_size
0,2011-01,141.504890
1,2011-02,143.110800
2,2011-03,139.375860
3,2011-04,138.844445
4,2011-05,140.768288
...,...,...
569,2021-48,240.514630
570,2021-49,240.011200
571,2021-50,228.710025
572,2021-51,230.274691


In [ ]:
# drop observations below median size
subset = CRSPstock.merge(temp,how='left',on='yearWeek')
todrop = subset.loc[subset['size']<=subset.cutoff_size,'PERMNO'].unique()
CRSPstock.drop(CRSPstock[CRSPstock.PERMNO.isin(todrop)].index,axis=0,inplace=True)

In [ ]:
save = pd.DataFrame()
for stock in CRSPstock.PERMNO.unique():
    reg_data = CRSPstock.loc[CRSPstock.PERMNO == stock,('yearWeek','lnAdiff','RET')]
    reg_data = reg_data.merge(df,on='yearWeek')
    reg_data.dropna(axis=0,inplace=True)

    if len(reg_data) < 52*8:
        continue

    y = reg_data['lnAdiff']
    X = reg_data[['lead_ETF','ETF_lnAdiff','lag_ETF',
                  'lead_ownership','ownership_lnAdiff','lag_ownership',
                  'lead_MKT','MKT_lnAdiff','lag_MKT',
                  'lead_RET','MKT_RET','lag_RET',
                  'RET']]
    X.RET = np.square(X.RET)
    X = sm.add_constant(X)
    model = sm.OLS(y, X).fit()
    
    save[stock] = [stock] + list(model.params[1:].values)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


In [ ]:
save = save.transpose()
save.columns = ['PERMNO','lead_ETF','ETF','lag_ETF',
                'lead_ownership','ownership','lag_ownership',
                'lead_MKT_LIQ','MKT_LIQ','lag_MKT_LIQ','lead_MKT_RET','MKT_RET','lag_MKT_RET','squared_RET']

In [ ]:
for var in save.columns[1:]:
    save[var] = save[var].astype(float)

In [ ]:
save.describe()

,lead_ETF,ETF,lag_ETF,lead_ownership,ownership,lag_ownership,lead_MKT_LIQ,MKT_LIQ,lag_MKT_LIQ,lead_MKT_RET,MKT_RET,lag_MKT_RET,squared_RET
count,2199.000000,2199.000000,2199.000000,2199.000000,2199.000000,2199.000000,2199.000000,2199.000000,2199.000000,2199.000000,2199.000000,2199.000000,2199.000000
mean,0.051400,-0.006536,0.145608,-0.150572,0.399763,-0.132809,0.319235,4.258604,-0.020884,-0.007269,0.408040,0.509991,68.954425
std,1.550483,1.904901,1.576496,1.735539,2.384122,1.589981,4.586661,6.923611,4.320623,2.015362,2.098861,2.211820,331.309983
min,-17.599750,-27.966567,-37.413915,-24.909129,-54.802550,-19.992984,-56.876408,-43.916251,-56.334752,-27.833588,-16.051704,-13.659573,-75.379393
25%,-0.274002,-0.238577,-0.154980,-0.389665,-0.079959,-0.353310,-0.370920,0.332489,-0.697239,-0.353016,-0.126078,-0.121692,5.287189
50%,-0.019405,0.043902,0.005414,-0.018218,0.045396,-0.009662,0.082371,1.797030,0.008862,-0.009662,0.095695,0.086708,19.089091
75%,0.176854,0.376635,0.319046,0.122967,0.679418,0.177989,1.150425,5.734866,0.742052,0.364699,0.834020,0.940997,63.494451
max,26.444159,19.104235,11.883269,17.895298,39.181220,16.586658,86.772810,108.982330,94.538963,29.238861,19.490568,15.136393,13319.756215


In [ ]:
for var in ['ETF','ownership','MKT_LIQ']:
    ic([var, ttest_1samp(save[var],popmean=0)[0]])

ic| [var, ttest_1samp(save[var],popmean=0)[0]]: ['ETF', -0.16088760977149288]
ic| [var, ttest_1samp(save[var],popmean=0)[0]]: ['ownership', 7.862972364029391]
ic| [var, ttest_1samp(save[var],popmean=0)[0]]: ['MKT_LIQ', 28.843446950384493]


### Robustness: ln(OCAmihud+1)

In [ ]:
with open('/content/drive/My Drive/Projects/Illiquidity_Contagion/Data/Raw/CRSPstock_week.pkl', 'rb') as f:
    CRSPstock = pickle.load(f)

In [ ]:
todrop = CRSPstock.loc[CRSPstock.PRC<2,'PERMNO'].unique()
CRSPstock.drop(CRSPstock[CRSPstock.PERMNO.isin(todrop)].index,axis=0,inplace=True)

In [ ]:
CRSPstock.lnOCAdiff = winsorization(CRSPstock.lnOCAdiff, 0.01)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 567 entries, 0 to 566
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   yearWeek             567 non-null    object 
 1   MKT_RET              567 non-null    float64
 2   MKT_BASdiff          567 non-null    float64
 3   MKT_lnAdiff          567 non-null    float64
 4   MKT_lnOCAdiff        567 non-null    float64
 5   ownership_BASdiff    567 non-null    float64
 6   ownership_lnAdiff    567 non-null    float64
 7   ownership_lnOCAdiff  567 non-null    float64
 8   ETF_BASdiff          567 non-null    float64
 9   ETF_lnAdiff          567 non-null    float64
 10  ETF_lnOCAdiff        567 non-null    float64
dtypes: float64(10), object(1)
memory usage: 53.2+ KB


In [ ]:
df['lead_MKT'] = df.MKT_lnOCAdiff.shift(-1)
df['lag_MKT'] = df.MKT_lnOCAdiff.shift(1)
df['lead_ownership'] = df.ownership_lnOCAdiff.shift(-1)
df['lag_ownership'] = df.ownership_lnOCAdiff.shift(1)
df['lead_ETF'] = df.ETF_lnOCAdiff.shift(-1)
df['lag_ETF'] = df.ETF_lnOCAdiff.shift(1)
df['lead_RET'] = df.MKT_RET.shift(-1)
df['lag_RET'] = df.MKT_RET.shift(1)
df.dropna(axis=0,inplace=True)

In [ ]:
save = pd.DataFrame()
for stock in CRSPstock.PERMNO.unique():
    reg_data = CRSPstock.loc[CRSPstock.PERMNO == stock,('yearWeek','lnOCAdiff','RET')]
    reg_data = reg_data.merge(df,on='yearWeek')
    reg_data.dropna(axis=0,inplace=True)

    if len(reg_data) < 52*8:
        continue

    y = reg_data['lnOCAdiff']
    X = reg_data[['lead_ETF','ETF_lnOCAdiff','lag_ETF',
                  'lead_ownership','ownership_lnOCAdiff','lag_ownership',
                  'lead_MKT','MKT_lnOCAdiff','lag_MKT',
                  'lead_RET','MKT_RET','lag_RET',
                  'RET']]
    X.RET = np.square(X.RET)
    X = sm.add_constant(X)
    model = sm.OLS(y, X).fit()
    
    save[stock] = [stock] + list(model.params[1:].values)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


In [ ]:
save = save.transpose()
save.columns = ['PERMNO','lead_ETF','ETF','lag_ETF',
                'lead_ownership','ownership','lag_ownership',
                'lead_MKT_LIQ','MKT_LIQ','lag_MKT_LIQ','lead_MKT_RET','MKT_RET','lag_MKT_RET','squared_RET']

In [ ]:
for var in save.columns[1:]:
    save[var] = save[var].astype(float)

In [ ]:
save.describe(percentiles = [0.0001,0.001,0.01,0.99,0.999,0.9999])

,lead_ETF,ETF,lag_ETF,lead_ownership,ownership,lag_ownership,lead_MKT_LIQ,MKT_LIQ,lag_MKT_LIQ,lead_MKT_RET,MKT_RET,lag_MKT_RET,squared_RET
count,2812.000000,2812.000000,2812.000000,2812.000000,2812.000000,2812.000000,2812.000000,2812.000000,2812.000000,2812.000000,2812.000000,2812.000000,2812.000000
mean,0.284055,-0.309378,0.313268,0.000242,-0.056705,0.015972,-0.479858,7.335465,-0.457176,-0.084234,0.009401,0.407961,64.276970
std,3.652625,3.982775,3.608176,1.810096,1.897314,1.755415,7.587637,10.841248,7.235465,3.685379,4.231660,4.256580,235.253541
min,-32.734855,-29.573488,-25.321553,-39.559871,-53.710084,-15.338325,-51.162588,-37.307400,-52.745494,-25.539721,-34.627052,-31.129281,-3010.936548
0.01%,-31.868747,-29.521445,-24.865009,-33.008569,-41.612031,-14.336127,-51.050981,-37.298735,-51.930115,-25.205263,-34.535158,-31.010421,-2287.039660
0.1%,-22.453860,-27.123007,-21.854590,-10.512324,-9.353727,-10.839932,-43.555156,-33.643575,-41.393821,-22.293741,-27.780762,-28.832298,-258.910293
1%,-10.808545,-14.300198,-11.536431,-6.252099,-6.713299,-6.647427,-25.408189,-11.185767,-25.166092,-13.057182,-15.101535,-13.393078,-59.976978
50%,0.003380,0.030725,-0.002338,-0.007481,0.022352,-0.001965,-0.008226,3.197004,0.000288,-0.002850,0.045285,0.027964,14.530289
99%,12.924419,12.035655,14.293218,6.249533,5.069125,6.939964,22.274076,46.826000,21.607857,12.700865,13.457660,14.516230,613.776126
99.9%,22.387004,20.755642,22.252593,9.839651,9.665397,10.259310,52.629669,72.541739,55.168194,23.415639,26.440786,25.211096,2851.032216


In [ ]:
for var in ['ETF','ownership','MKT_LIQ']:
    ic([var, ttest_1samp(save[var],popmean=0)[0]])

ic| [var, ttest_1samp(save[var],popmean=0)[0]]: ['ETF', -4.119180556262842]
ic| [var, ttest_1samp(save[var],popmean=0)[0]]: ['ownership', -1.5848529270574543]
ic| [var, ttest_1samp(save[var],popmean=0)[0]]: ['MKT_LIQ', 35.88029911386288]


Top50% subsample

In [ ]:
with open('/content/drive/My Drive/Projects/Illiquidity_Contagion/Data/Raw/CRSPstock_week.pkl', 'rb') as f:
    CRSPstock = pickle.load(f)

In [ ]:
# get the size median over time
temp = CRSPstock[["yearWeek","size"]].groupby(["yearWeek"]).quantile(0.5).reset_index()
temp.columns = ["yearWeek","cutoff_size"]
temp

,yearWeek,cutoff_size
0,2011-01,504.076200
1,2011-02,507.213370
2,2011-03,495.541500
3,2011-04,491.422658
4,2011-05,502.099200
...,...,...
569,2021-48,1049.175480
570,2021-49,1063.543990
571,2021-50,1022.392160
572,2021-51,1033.035400


In [ ]:
# drop observations below median size
subset = CRSPstock.merge(temp,how='left',on='yearWeek')
todrop = subset.loc[subset['size']<=subset.cutoff_size,'PERMNO'].unique()
CRSPstock.drop(CRSPstock[CRSPstock.PERMNO.isin(todrop)].index,axis=0,inplace=True)

In [ ]:
save = pd.DataFrame()
for stock in CRSPstock.PERMNO.unique():
    reg_data = CRSPstock.loc[CRSPstock.PERMNO == stock,('yearWeek','lnOCAdiff','RET')]
    reg_data = reg_data.merge(df,on='yearWeek')
    reg_data.dropna(axis=0,inplace=True)

    if len(reg_data) < 52*8:
        continue

    y = reg_data['lnOCAdiff']
    X = reg_data[['lead_ETF','ETF_lnOCAdiff','lag_ETF',
                  'lead_ownership','ownership_lnOCAdiff','lag_ownership',
                  'lead_MKT','MKT_lnOCAdiff','lag_MKT',
                  'lead_RET','MKT_RET','lag_RET',
                  'RET']]
    X.RET = np.square(X.RET)
    X = sm.add_constant(X)
    model = sm.OLS(y, X).fit()
    
    save[stock] = [stock] + list(model.params[1:].values)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


In [ ]:
save = save.transpose()
save.columns = ['PERMNO','lead_ETF','ETF','lag_ETF',
                'lead_ownership','ownership','lag_ownership',
                'lead_MKT_LIQ','MKT_LIQ','lag_MKT_LIQ','lead_MKT_RET','MKT_RET','lag_MKT_RET','squared_RET']

In [ ]:
for var in save.columns[1:]:
    save[var] = save[var].astype(float)

In [ ]:
save.describe(percentiles = [0.0001,0.001,0.01,0.99,0.999,0.9999])

,lead_ETF,ETF,lag_ETF,lead_ownership,ownership,lag_ownership,lead_MKT_LIQ,MKT_LIQ,lag_MKT_LIQ,lead_MKT_RET,MKT_RET,lag_MKT_RET,squared_RET
count,1366.000000,1366.000000,1366.000000,1366.000000,1366.000000,1366.000000,1366.000000,1366.000000,1366.000000,1366.000000,1366.000000,1366.000000,1366.000000
mean,0.031493,0.120413,0.002485,-0.025436,0.060579,-0.024985,-0.089649,2.113335,-0.038065,-0.022670,0.196995,0.156242,18.911242
std,0.668262,0.852054,0.779520,0.302829,0.351735,0.346087,1.402712,3.454434,1.668588,0.879501,0.893917,0.896168,108.892016
min,-4.090387,-9.548654,-12.293600,-6.756591,-8.006194,-6.390715,-12.346611,-4.380124,-24.113721,-4.929607,-7.857128,-3.922643,-51.401895
0.01%,-4.052708,-9.281031,-11.724718,-6.073619,-7.258735,-6.174312,-12.258977,-4.127334,-22.238808,-4.862764,-7.587652,-3.903999,-50.829180
0.1%,-3.478792,-7.482426,-7.645709,-1.580234,-2.297099,-4.381905,-11.665624,-2.290262,-9.716928,-4.124807,-5.469607,-3.743407,-45.221861
1%,-1.725851,-2.326575,-1.927571,-0.693487,-0.690808,-0.738577,-4.871820,-0.213072,-4.474872,-2.238194,-2.117324,-1.949113,-6.147391
50%,-0.003583,0.058552,-0.010140,-0.006167,0.018484,-0.002514,0.002063,0.883228,0.007485,-0.002277,0.044142,0.006744,4.229332
99%,2.515654,2.969464,2.282388,0.562674,0.957897,0.538440,3.859997,17.256067,4.203318,1.743650,3.123534,3.801574,227.068671
99.9%,5.705604,5.627389,4.704790,2.514350,1.611375,2.545960,6.849075,31.712996,16.582059,11.567218,5.979140,8.988153,1216.454425


In [ ]:
for var in ['ETF','ownership','MKT_LIQ']:
    ic([var, ttest_1samp(save[var],popmean=0)[0]])

ic| [var, ttest_1samp(save[var],popmean=0)[0]]: ['ETF', 5.2231233136046225]
ic| [var, ttest_1samp(save[var],popmean=0)[0]]: ['ownership', 6.36552649739027]
ic| [var, ttest_1samp(save[var],popmean=0)[0]]: ['MKT_LIQ', 22.61085115510315]
